# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(64, 64, 3)
BATCH_SIZE = 32
latent_dim = 100
intermediate_dim = 100
TRAIN_BUF = 600
TEST_BUF = 100

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

12288

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AAE'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.adversarial.AAE import AAE as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAAE\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 3)]       0         
_________________________________________________________________
dense (Dense)                (None, 64, 64, 100)       400       
_________________________________________________________________
dense_1 (Dense)              (None, 64, 64, 100)       10100     
_________________________________________________________________
flatten (Flatten)            (None, 409600)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               40960100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 12288)             1241088   
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 3)         0

  WARNING  | None


In [26]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAAE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 64, 64, 3)]       0         
_________________________________________________________________
inference (Model)            (None, 100)               40971000  
_________________________________________________________________
generative (Model)           (None, 64, 64, 3)         1261300   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 64, 64, 3)]       0         
Total params: 42,232,300
Trainable params: 42,232,094
Non-trainable params: 206
_________________________________________________________________
None


# Callbacks

In [27]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [28]:
progbar = NotebookPrograssBar(leave_outer=False)

In [29]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [30]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [31]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAAE\\csv_dir\\pokemonAAE.csv'

In [32]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [33]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [34]:
import numpy as np

# Model Training

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=50,
    epochs=2, #int(1e6)
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True
)

training traditional basicAE


Model: "latent_real_discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 12288)             1241088   
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 3)         12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 6

50/50 [==============================] - ETA: 8s - loss: 0.025 - ETA: 8s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 7s - loss: 0.025 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 4s - loss: 0.024 - ETA: 4s - loss: 0.024 - ETA: 4s - loss: 0.024 - ETA: 4s - loss: 0.024 - ETA: 4s - loss: 0.024 - ETA: 4s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 1s - loss: 0.023 - ETA: 1s - loss: 0.023 -

Epoch 1/2


36/50 [====================>.........] - ETA: 3:03 - loss: 5020.1182 - latent_fake_discriminator_outputs_loss: 19.2961 - latent_real_discriminator_outputs_loss: 1.5360 - x_logits_loss: 4999.1797 - x_logits_psnr: 8.9132 - x_logits_ssmi: 0.0080 - x_logits_sharp_diff: 4.90 - ETA: 1:34 - loss: 5147.1565 - latent_fake_discriminator_outputs_loss: 14.5769 - latent_real_discriminator_outputs_loss: 1.2684 - x_logits_loss: 5131.2051 - x_logits_psnr: 8.8123 - x_logits_ssmi: 0.0077 - x_logits_sharp_diff: 4.90 - ETA: 1:05 - loss: 5193.5021 - latent_fake_discriminator_outputs_loss: 10.2819 - latent_real_discriminator_outputs_loss: 1.2896 - x_logits_loss: 5181.8242 - x_logits_psnr: 8.7406 - x_logits_ssmi: 0.0079 - x_logits_sharp_diff: 4.90 - ETA: 50s - loss: 5151.6461 - latent_fake_discriminator_outputs_loss: 7.8213 - latent_real_discriminator_outputs_loss: 1.1397 - x_logits_loss: 5142.5786 - x_logits_psnr: 8.7686 - x_logits_ssmi: 0.0080 - x_logits_sharp_diff: 4.9140 - ETA: 40s - loss: 5071.5814 - la

49/50 [============================>.] - ETA: 3s - loss: 5022.1155 - latent_fake_discriminator_outputs_loss: 0.8721 - latent_real_discriminator_outputs_loss: 0.1533 - x_logits_loss: 5020.9829 - x_logits_psnr: 8.8896 - x_logits_ssmi: 0.0088 - x_logits_sharp_diff: 4.954 - ETA: 3s - loss: 5025.1555 - latent_fake_discriminator_outputs_loss: 0.8507 - latent_real_discriminator_outputs_loss: 0.1492 - x_logits_loss: 5024.0483 - x_logits_psnr: 8.8856 - x_logits_ssmi: 0.0088 - x_logits_sharp_diff: 4.955 - ETA: 3s - loss: 5024.2637 - latent_fake_discriminator_outputs_loss: 0.8304 - latent_real_discriminator_outputs_loss: 0.1454 - x_logits_loss: 5023.1807 - x_logits_psnr: 8.8848 - x_logits_ssmi: 0.0088 - x_logits_sharp_diff: 4.956 - ETA: 2s - loss: 5021.6213 - latent_fake_discriminator_outputs_loss: 0.8096 - latent_real_discriminator_outputs_loss: 0.1418 - x_logits_loss: 5020.5625 - x_logits_psnr: 8.8849 - x_logits_ssmi: 0.0088 - x_logits_sharp_diff: 4.957 - ETA: 2s - loss: 5013.9888 - latent_fake

KeyError: 'psnr'